### Carregamento

In [1]:
import pandas as pd
# Função para carregar os dados
def carregar_dados(caminho_arquivo, coluna_data=None):
    """
    Carrega um arquivo CSV com dados.
    
    Parâmetros:
    - caminho_arquivo (str): Caminho do arquivo CSV.
    - coluna_data (str, opcional): Nome da coluna de datas para converter para datetime.
    
    Retorna:
    - pandas.DataFrame: DataFrame com os dados carregados.
    """
    # Carrega o CSV com separador ';'
    dados = pd.read_csv(caminho_arquivo, sep=',')
    
    # Converte a coluna de datas para datetime com timezone, se especificada
    if coluna_data:
        dados[coluna_data] = pd.to_datetime(dados[coluna_data], utc=True)
    
    return dados
# Caminho
caminho_arquivo = '../data/raw/salary.csv'
# Carrega os dados com o separador correto
dados = carregar_dados(caminho_arquivo)

# Exibir as primeiras linhas do DataFrame
dados.head(10)


,Position,Level,Salary
0,Business Analyst,1,45000
1,Junior Consultant,2,50000
2,Senior Consultant,3,60000
3,Manager,4,80000
4,Country Manger,5,110000
5,Region Manager,6,150000
6,Partner,7,200000
7,Senior Partner,8,220000
8,C.level,9,250000
9,CEO,10,1000000


### Tamanho base

In [2]:
def verificar_tamanho_base(df):
    """
    Verifica o tamanho da base de dados em termos de número de linhas e colunas.
    
    Parâmetros:
    - df (pandas.DataFrame): DataFrame a ser verificado.
    
    Retorna:
    - dict: Dicionário com o número de linhas e colunas.
    """
    tamanho = {
        'Número de Linhas': df.shape[0],
        'Número de Colunas': df.shape[1]
    }
    
    return tamanho

# Verificar o tamanho da base de dados
tamanho_base = verificar_tamanho_base(dados)
tamanho_base

{'Número de Linhas': 10, 'Número de Colunas': 3}

### Verificar nulos

In [3]:
# Função para verificar valores ausentes
def verificar_valores_ausentes(df):
    """
    Verifica a quantidade e o percentual de valores ausentes em cada coluna de um DataFrame.
    
    Parâmetros:
    - df (pandas.DataFrame): DataFrame a ser verificado.
    
    Retorna:
    - pandas.DataFrame: DataFrame com a quantidade e o percentual de valores ausentes por coluna.
    """
    # Calcular a quantidade de valores ausentes
    valores_ausentes = df.isnull().sum()
    
    # Calcular o percentual de valores ausentes
    percentual_ausentes = (valores_ausentes / df.shape[0]) * 100
    
    # Criar um DataFrame com os resultados
    tabela_ausentes = pd.DataFrame({
        'Quantidade de Valores Ausentes': valores_ausentes,
        'Percentual de Valores Ausentes (%)': percentual_ausentes
    })
    
    # Ordenar o DataFrame pelo percentual de valores ausentes em ordem decrescente
    tabela_ausentes = tabela_ausentes[tabela_ausentes['Quantidade de Valores Ausentes'] > 0]
    tabela_ausentes = tabela_ausentes.sort_values(by='Percentual de Valores Ausentes (%)', ascending=False)
    
    return tabela_ausentes

# Verificar valores ausentes
tabela_ausentes = verificar_valores_ausentes(dados)
tabela_ausentes

,Quantidade de Valores Ausentes,Percentual de Valores Ausentes (%)


### Valores duplicados

In [15]:
def verificar_valores_duplicados(df, colunas=None):
    """
    Verifica valores duplicados em um DataFrame, fornecendo análise detalhada.
    
    Parâmetros:
    - df (pandas.DataFrame): DataFrame a ser analisado
    - colunas (list, opcional): Lista específica de colunas para verificar duplicatas.
                               Se None, verifica todas as colunas.
    
    Retorna:
    - dict: Dicionário com informações sobre duplicatas para cada coluna analisada
    """
    resultado = {}
    
    # Se nenhuma coluna específica for fornecida, usar todas as colunas
    if colunas is None:
        colunas = df.columns
    
    # Análise de duplicatas por coluna
    for coluna in colunas:
        # Encontrar valores duplicados
        duplicados = df[df[coluna].duplicated(keep='first')]
        qtd_duplicados = len(duplicados)
        
        if qtd_duplicados > 0:
            # Contagem de ocorrências de cada valor duplicado
            valores_duplicados = df[coluna].value_counts()[df[coluna].value_counts() > 1]
            
            resultado[coluna] = {
                'possui_duplicados': True,
                'quantidade_duplicados': qtd_duplicados,
                'valores_e_contagem': valores_duplicados.to_dict()
            }
        else:
            resultado[coluna] = {
                'possui_duplicados': False,
                'quantidade_duplicados': 0,
                'valores_e_contagem': {}
            }
    
    # Exibir resumo
    print("=== Análise de Valores Duplicados ===\n")
    
    for coluna, info in resultado.items():
        print(f"\nColuna: {coluna}")
        if info['possui_duplicados']:
            print(f"Status: Possui {info['quantidade_duplicados']} valores duplicados")
            print("Valores duplicados e suas ocorrências:")
            for valor, contagem in info['valores_e_contagem'].items():
                print(f"  - Valor: {valor} | Ocorrências: {contagem}")
        else:
            print("Status: Não possui valores duplicados")
    
    return resultado

# Executar verificação de valores duplicados
resultados_duplicados = verificar_valores_duplicados(dados)

=== Análise de Valores Duplicados ===


Coluna: Position
Status: Não possui valores duplicados

Coluna: Level
Status: Não possui valores duplicados

Coluna: Salary
Status: Não possui valores duplicados


### Verificação tipos dos dados

In [13]:
# Função para verificar o tipo dos dados/colunas
def verificar_tipo_dados(df):
    """
    Verifica o tipo dos dados/colunas de um DataFrame.
    
    Parâmetros:
    - df (pandas.DataFrame): DataFrame a ser verificado.
    
    Retorna:
    - pandas.DataFrame: DataFrame com os nomes das colunas e seus respectivos tipos de dados.
    """
    # Obter os tipos de dados das colunas
    tipos_dados = df.dtypes
    
    # Criar um DataFrame com os resultados
    tabela_tipos = pd.DataFrame({
        'Coluna': tipos_dados.index,
        'Tipo de Dado': tipos_dados.values
    })
    
    return tabela_tipos

# Verificar o tipo dos dados/colunas
tabela_tipos = verificar_tipo_dados(dados)
tabela_tipos

,Coluna,Tipo de Dado
0,Position,object
1,Level,int64
2,Salary,int64


### Eliminar possíveis espaços

In [19]:
def remover_espacos(df):
    """
    Remove espaços em branco à esquerda e à direita das strings em todas as colunas e nos nomes das colunas.
    
    Parâmetros:
    - df (pandas.DataFrame): DataFrame a ser processado.
    
    Retorna:
    - pandas.DataFrame: DataFrame com os espaços removidos.
    - pandas.DataFrame: DataFrame com informações sobre as remoções realizadas.
    """
    # Verificar se o DataFrame é válido
    if not isinstance(df, pd.DataFrame):
        raise ValueError("O argumento fornecido não é um DataFrame.")
    
    # Verificar espaços nos nomes das colunas antes
    colunas_com_espacos = [col for col in df.columns if col != col.strip()]
    espacos_colunas_antes = len(colunas_com_espacos)
    
    # Remover espaços dos nomes das colunas
    df.columns = df.columns.str.strip()
            
    # Inicializar lista para armazenar informações sobre remoções
    informacoes_remocoes = []

    # Adicionar informação sobre limpeza dos nomes das colunas
    informacoes_remocoes.append({
        'Coluna': 'NOMES DAS COLUNAS',
        'Espaços Antes': espacos_colunas_antes,
        'Espaços Depois': 0,
        'Remoção Bem-Sucedida': True
    })

    # Aplicar strip em todas as colunas de tipo object e category
    for col in df.select_dtypes(include=['object', 'category']).columns:
        # Contar espaços em branco antes da remoção
        espacos_antes = df[col].apply(lambda x: x != x.strip() if isinstance(x, str) else False).sum()
        
        # Remover espaços em branco
        df[col] = df[col].map(lambda x: x.strip() if isinstance(x, str) else x)
        
        # Contar espaços em branco após a remoção
        espacos_depois = df[col].apply(lambda x: x != x.strip() if isinstance(x, str) else False).sum()
        
        # Verificar se a remoção foi bem-sucedida
        sucesso = espacos_depois == 0
        
        # Adicionar informações à lista
        informacoes_remocoes.append({
            'Coluna': col,
            'Espaços Antes': espacos_antes,
            'Espaços Depois': espacos_depois,
            'Remoção Bem-Sucedida': sucesso
        })
    
    # Criar DataFrame com as informações de remoção
    df_informacoes_remocoes = pd.DataFrame(informacoes_remocoes)
    
    return df, df_informacoes_remocoes

# Exemplo de uso:
dados, informacoes_remocoes = remover_espacos(dados)

# Exibir as informações sobre as remoções
print("Informações sobre a remoção de espaços em branco:")
display(informacoes_remocoes)

Informações sobre a remoção de espaços em branco:


,Coluna,Espaços Antes,Espaços Depois,Remoção Bem-Sucedida
0,NOMES DAS COLUNAS,0,0,True
1,Position,0,0,True


### Conversão otimização

In [20]:
def modificar_tipo_colunas(df, colunas_tipos):
    """
    Modifica o tipo de dado de colunas específicas em um DataFrame.
    
    Parâmetros:
    - df (pandas.DataFrame): DataFrame original.
    - colunas_tipos (dict): Dicionário onde as chaves são os nomes das colunas e os valores são os tipos de dados desejados.
    
    Retorna:
    - pandas.DataFrame: DataFrame com os tipos de dados modificados.
    """
    # Verifica se as colunas informadas existem no DataFrame
    colunas_invalidas = [col for col in colunas_tipos.keys() if col not in df.columns]
    if colunas_invalidas:
        raise ValueError(f"Colunas inválidas: {colunas_invalidas}")
    
    # Modifica o tipo de dado das colunas especificadas
    for coluna, tipo in colunas_tipos.items():
        try:
            df[coluna] = df[coluna].astype(tipo)
            print(f"✅ Coluna '{coluna}' convertida para '{tipo}' com sucesso.")
        except Exception as e:
            print(f"⚠️ Erro ao converter a coluna '{coluna}' para '{tipo}': {e}")
    
    return df

# Exemplo de uso:
colunas_tipos = {
    'Position': 'category',
    'Level': 'int8',
    'Salary': 'int32'
}
 
dados = modificar_tipo_colunas(dados, colunas_tipos)
dados.dtypes # Verificar os tipos de dados


✅ Coluna 'Position' convertida para 'category' com sucesso.
✅ Coluna 'Level' convertida para 'int8' com sucesso.
✅ Coluna 'Salary' convertida para 'int32' com sucesso.


Position    category
Level           int8
Salary         int32
dtype: object

### Carga

In [21]:
def salvar_dados(df, caminho_arquivo_saida):
    """
    Salva o DataFrame em um arquivo CSV.
    
    Parâmetros:
    - df (pandas.DataFrame): DataFrame a ser salvo.
    - caminho_arquivo_saida (str): Caminho do arquivo CSV de saída.
    """
    df.to_csv(caminho_arquivo_saida, index=False)
    print(f"✅ Dados salvos com sucesso em '{caminho_arquivo_saida}'")

# Exemplo de uso:
caminho_arquivo_saida = '../data/processed/salary_atualizado.csv'
salvar_dados(dados, caminho_arquivo_saida)

✅ Dados salvos com sucesso em '../data/processed/salary_atualizado.csv'
